In [56]:
# part of power analysis function #
# assuming I can get an array of SEs #

#import mne
import numpy as np
import pandas as pd
import os
import glob
from matplotlib import pyplot as plt
import sys
import warnings
import seaborn as sns
from itertools import product as prod

# temporary 
from scipy import stats
from statsmodels.stats.power import ttest_power, tt_ind_solve_power



In [210]:
# create arrays of SEs, using 4 electrodes

arr_ele = np.array([.01,1,2,10])

# fake subjects and effect sizes
Subs = [20,25,30,35]
effect = [3, 5]


In [211]:
# t tests with simple array

t_arr = np.zeros((len(arr_ele), len(Subs), len(effect)))
p_arr = np.zeros((len(arr_ele), len(Subs), len(effect)))

for ele in range(len(arr_ele)):  # loop over electrodes
    BESE = arr_simp[ele]
    
    for s in range(len(Subs)):    # loop over sample sizes
        n = Subs[s]
        df = 2*n - 2              # get degrees of freedom (based on sample size)
        
        for ef in range(len(effect)):          # loop over effect sizes
            t = effect[ef] / (BESE/np.sqrt(n)) # get t value 
            t_arr[ele][s][ef] = t              # save t value
            
            p = 1 - stats.t.cdf(t,df=df)       # get p value
            p_arr[ele][s][ef] = p              # save p value


In [212]:
# t values
print(t_arr)

[[[1.34164079e+03 2.23606798e+03]
  [1.50000000e+03 2.50000000e+03]
  [1.64316767e+03 2.73861279e+03]
  [1.77482393e+03 2.95803989e+03]]

 [[1.34164079e+01 2.23606798e+01]
  [1.50000000e+01 2.50000000e+01]
  [1.64316767e+01 2.73861279e+01]
  [1.77482393e+01 2.95803989e+01]]

 [[6.70820393e+00 1.11803399e+01]
  [7.50000000e+00 1.25000000e+01]
  [8.21583836e+00 1.36930639e+01]
  [8.87411967e+00 1.47901995e+01]]

 [[1.34164079e+00 2.23606798e+00]
  [1.50000000e+00 2.50000000e+00]
  [1.64316767e+00 2.73861279e+00]
  [1.77482393e+00 2.95803989e+00]]]


In [213]:
# p values
print ( p_arr)

[[[0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00]]

 [[3.33066907e-16 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00]]

 [[3.06019163e-08 7.02771175e-14]
  [6.32174757e-10 1.11022302e-16]
  [1.33266731e-11 0.00000000e+00]
  [2.84883228e-13 0.00000000e+00]]

 [[9.38377405e-02 1.56487342e-02]
  [7.00818569e-02 7.94484547e-03]
  [5.28795835e-02 4.09079077e-03]
  [4.02012659e-02 2.12778133e-03]]]


In [215]:
# fake arrays

Subs = [1,10,20,50]
effects = [1, 2, 3, 5]
arr_ele = np.array([1,2,3,5])


In [216]:
pwrs = np.zeros((len(arr_ele), len(Subs), len(effects)))

for ele in range(len(arr_ele)):  # loop over electrodes
    BESE = arr_simp[ele]
    
    for s in range(len(Subs)):    # loop over sample sizes
        n = Subs[s]
        SD = BESE/(np.sqrt(n))    # get SD from BESE (maybe wrong)
        #null_dist = np.random.normal(loc = 0, scale = SD, size = 1000)     # make null distribution
        crit = 1.96*SD                                                      # get critical value
        
        for effect in range(len(effects)):                                  # loop over effect sizes
            ef = effects[effect]
            alt_dist = np.random.normal(loc = ef, scale = SD, size = 10000) # make alternative distribution 
            pwr = len(alt_dist[alt_dist>crit])/len(alt_dist)
            
            pwrs[ele][s][effect] = pwr              # save power 


In [217]:
pwrs

array([[[0.1691, 0.521 , 0.8568, 0.9991],
        [0.8871, 1.    , 1.    , 1.    ],
        [0.994 , 1.    , 1.    , 1.    ],
        [1.    , 1.    , 1.    , 1.    ]],

       [[0.0727, 0.1648, 0.3229, 0.7146],
        [0.3586, 0.8831, 0.9984, 1.    ],
        [0.6076, 0.9947, 1.    , 1.    ],
        [0.9425, 1.    , 1.    , 1.    ]],

       [[0.0475, 0.0963, 0.1751, 0.3805],
        [0.1844, 0.5604, 0.8867, 0.9996],
        [0.3154, 0.8448, 0.9951, 1.    ],
        [0.6541, 0.9975, 1.    , 1.    ]],

       [[0.0386, 0.0588, 0.0848, 0.175 ],
        [0.0923, 0.242 , 0.4765, 0.8848],
        [0.1431, 0.437 , 0.7632, 0.9946],
        [0.2884, 0.8076, 0.9887, 1.    ]]])

In [218]:
# now for the item option...

# here, we will have to make effect size be a constant

effect = 2
Subs = [1,10] # usually would be 4

# sample array
arr_item = np.array([[1,2,3,4],[2,4,6,8],[3,6,9,12]]) # rows are items(usually would be 4), columns are electrodes
arr_item

array([[ 1,  2,  3,  4],
       [ 2,  4,  6,  8],
       [ 3,  6,  9, 12]])

In [219]:
pwrs_i = np.zeros((arr_item.shape[1], arr_item.shape[0], len(Subs)))

for ele in range(arr_item.shape[1]):  # loop over electrodes
    
    for item in range(arr_item.shape[0]):    # loop over item samples
        BESE = arr_item[item][ele]
          
        for s in range(len(Subs)):               # loop over subject samples
            n = Subs[s]
            SD = BESE/(np.sqrt(n))                                              # get SD from BESE (maybe wrong)
            crit = 1.96*SD                                                      # get critical value
            alt_dist = np.random.normal(loc = effect, scale = SD, size = 10000) # make alternative distribution 
            pwr_i = len(alt_dist[alt_dist>crit])/len(alt_dist)                  # get power
            
            pwrs_i[ele][item][s] = pwr_i                                        # store powers

In [220]:
pwrs_i # z = electrodes, y = subs, x = item samples

array([[[0.5169, 1.    ],
        [0.1642, 0.8839],
        [0.0959, 0.5561]],

       [[0.1641, 0.8805],
        [0.0694, 0.3477],
        [0.0521, 0.1753]],

       [[0.0964, 0.5646],
        [0.0514, 0.1849],
        [0.0409, 0.1019]],

       [[0.0703, 0.3644],
        [0.0443, 0.128 ],
        [0.0343, 0.0736]]])